Examples require an initialized GsSession and relevant entitlements. `run_analytics` scope is required for the functionality covered in this tutorial. External clients need to substitute thier own client id and client secret below. Please refer to <a href="https://developer.gs.com/docs/gsquant/guides/Authentication/2-gs-session/"> Sessions</a> for details.

In [ ]:
from gs_quant.session import GsSession
GsSession.use(client_id=None, client_secret=None, scopes=('run_analytics',))

This tutorial demonstrates how to use `gs-quant` to price and calculate risk historically. We will
use the swaption instrument definition from the
[first tutorial](https://developer.gs.com/docs/gsquant/tutorials/Pricing-and-Risk/1-creating-an-instrument/).

In [ ]:
from gs_quant.instrument import IRSwaption
from gs_quant.common import PayReceive, Currency

swaption = IRSwaption(PayReceive.Receive, '5y', Currency.USD, expiration_date='13m', strike='atm+40', notional_amount=1e8)

## Historical Pricing

In the previous tutorial, we priced the instrument spot. Let's now use
[`HistoricalPricingContext`](https://developer.gs.com/docs/gsquant/api/classes/gs_quant.markets.HistoricalPricingContext.html)
to get daily prices for May 2019. See the
[pricing context guide](https://developer.gs.com/docs/gsquant/guides/Pricing-and-Risk/pricing-context/) guide for additional
details.


In [ ]:
from gs_quant.markets import HistoricalPricingContext
import datetime as dt

start_date = dt.date(2019, 5, 1)
end_date = dt.date(2019, 5, 31)

with HistoricalPricingContext(start_date, end_date):
    swaption_price = swaption.price()

swaption_prices = swaption_price.result()
swaption_prices

## Historical Risk

We can use the same
[`HistoricalPricingContext`](https://developer.gs.com/docs/gsquant/api/classes/gs_quant.markets.HistoricalPricingContext.html)
to calculate historical risk measures. Please refer to the
[measures guide](https://developer.gs.com/docs/gsquant/guides/Pricing-and-Risk/measures/) for a list of externally supported risk
measures.

Let's calculate the vega risk for the swaption above:

In [ ]:
with HistoricalPricingContext(start_date, end_date):
    swaption_vega = swaption.calc(risk.IRVega)

swaption_vegas = swaption_vega.result()
swaption_vegas.head() # limit to several rows

See the [pricing context guide](https://developer.gs.com/docs/gsquant/guides/Pricing-and-Risk/pricing-context/) for additional
details.

## Visualizing Results

Let's now put it all together and visualize the result:


In [ ]:
from gs_quant.common import PayReceive
from gs_quant.instrument import IRSwaption
from gs_quant.markets import HistoricalPricingContext
import datetime as dt
import gs_quant.risk as risk
import matplotlib.pyplot as plt

swaption = IRSwaption(PayReceive.Receive, '5y', 'USD', expirationDate='13m', strike='atm+40', notionalAmount=1e8)
swaption.resolve()

start_date = dt.date(2019, 5, 1)
end_date = dt.date(2019, 5, 31)

with HistoricalPricingContext(start_date, end_date):
    swaption_price = swaption.price()
    swaption_vega  = swaption.calc(risk.IRVega)

swaption_prices = swaption_price.result()
swaption_vegas  = swaption_vega.result()

# Plot prices over time
swaption_prices.plot(figsize=(10,6), title="Swaption Price Over Time")
plt.xlabel('Date')
plt.ylabel('Price')

# Group vegas by point and plot over time
pivot_by_point = swaption_vegas.pivot_table('value', index='date', columns ='point')
pivot_by_point.plot(figsize=(10,6),title='Vega Points Over Time')
plt.xlabel('Date')
plt.ylabel('Vega')

#### Disclaimer
This website may contain links to websites and the content of third parties ("Third Party Content"). We do not monitor, review or update, and do not have any control over, any Third Party Content or third party websites. We make no representation, warranty or guarantee as to the accuracy, completeness, timeliness or reliability of any Third Party Content and are not responsible for any loss or damage of any sort resulting from the use of, or for any failure of, products or services provided at or from a third party resource. If you use these links and the Third Party Content, you acknowledge that you are doing so entirely at your own risk.